In [ ]:
import numpy as np
import pickle
import os
import pyimfit
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import angular_separation
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from photutils.isophote import EllipseSample, Isophote
from photutils.isophote.sample import CentralEllipseSample
from photutils.isophote.fitter import CentralEllipseFitter
from photutils.isophote import Ellipse, EllipseGeometry, IsophoteList
from matplotlib.ticker import FormatStrFormatter
import glob
import matplotlib.gridspec as gridspec
from photutils.aperture import EllipticalAperture
from photutils.detection import find_peaks
import astropy.units as u
from astropy.cosmology import WMAP9 as cosmo
import seaborn as sns

sns.set_context("paper",font_scale=1.75)
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Times"],
    "text.latex.preamble": r"\usepackage{amsmath}\usepackage{mathptmx}",  # Times Roman
    "hatch.linewidth": 3.0,  
    "xtick.labelsize": 13,      
    "ytick.labelsize": 13,      
    "legend.fontsize": 13,

})

In [ ]:
def find_highest_indices(arr):
    """returns a tuple of ys, xs - indices of pixels with highest counts"""
    flattened_arr = np.array(arr).flatten()
    max_indices = np.unravel_index(np.argsort(flattened_arr)[-2:], arr.shape)
    return max_indices

def makeModelDict(PA_ss, ell_ss, n_ss, I_ss, r_ss, Itot,
                 PA_lim, ell_lim, I_lim,  Iss_lim, rss_lim, Itot_lim,
                 sigma, sigma_lim, Isky, Isky_lim,
                 h1,h2,h_lim,alpha,alpha_lim):
    """Return Sersic, PSF, and Gaussian model parameter dictionary"""
    # Sersic
    """sersic = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell_bulge': [ell_ss, ell_lim[0],ell_lim[1]], 'n': [n_ss, 'fixed'],
    'I_e': [I_ss, Iss_lim[0],Iss_lim[1]], 'r_e': [r_ss, rss_lim[0],rss_lim[1]]}"""
    sersic = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell_bulge': [ell_ss, ell_lim[0],ell_lim[1]], 'n': [n_ss, 0.3, 6],
    'I_e': [I_ss, Iss_lim[0],Iss_lim[1]], 'r_e': [r_ss, rss_lim[0],rss_lim[1]]}
    sersic_dict = {'name': "Sersic", 'label': "bulge", 'parameters': sersic}
    # PSF
    psf = {'I_tot' : [Itot, Itot_lim[0], Itot_lim[1]]}
    psf_dict = {'name': "PointSource", 'label': "psf", 'parameters': psf}
    # Rot psf
    psfRot = {'I_tot' : [Itot, Itot_lim[0], Itot_lim[1]], 'PA':[PA_ss, PA_lim[0],PA_lim[1]] }
    psfRot_dict = {'name': "PointSourceRot", 'label': "psf", 'parameters': psfRot}
    # Gaussians
    gaussian = {'PA':[PA_ss, PA_lim[0],PA_lim[1]], 'ell':[ell_ss, ell_lim[0],ell_lim[1]], 
                'I_0':[I_ss, Iss_lim[0],Iss_lim[1]], 'sigma':[sigma, sigma_lim[0], sigma_lim[1]]}
    gaussian_dict = {'name': "Gaussian", 'label': "gaussian", 'parameters': gaussian}
    # Flat sky
    flatsky = {'I_sky': [Isky, Isky_lim[0], Isky_lim[1]]}
    flatsky_dict = {'name': "FlatSky", 'label': "flat_sky", 'parameters':flatsky}
    # flat bar
    flatbar = {'PA':[PA_ss, PA_lim[0],PA_lim[1]], 'ell':[ell_ss, ell_lim[0],ell_lim[1]],
               'deltaPA_max':[PA_ss, PA_lim[0],PA_lim[1]], 'I_0':[I_ss, Iss_lim[0],Iss_lim[1]],
               'h1':[h1, h_lim[0],h_lim[1]], 'h2':[h2, h_lim[0],h_lim[1]], 
               'r_break':[r_ss, rss_lim[0],rss_lim[1]], 'alpha':[alpha,alpha_lim[0],alpha_lim[1]]}
    flatbar_dict = {'name': "FlatBar", 'label': "flat_bar", 'parameters':flatbar}
    # Exponential
    exponential = {'PA': [PA_ss, PA_lim[0],PA_lim[1]], 'ell': [ell_ss, ell_lim[0],ell_lim[1]], 
                   'I_0': [I_ss, Iss_lim[0],Iss_lim[1]], 'h': [h1, h_lim[0],h_lim[1]]}
    exp_dict = {'name': "Exponential", 'label': "disk", 'parameters':exponential}
    return sersic_dict, psf_dict, gaussian_dict, flatsky_dict, flatbar_dict,exp_dict

def find_sky(imageAGNcrop, plothist=False):
    bgr = plt.hist(imageAGNcrop.flatten(),bins=np.arange(np.min(imageAGNcrop),20))
    plt.close()
    max_ind = np.where(bgr[0]==np.max(bgr[0]))[0][0]
    sky = (bgr[1][max_ind]+bgr[1][max_ind+1])/2
    if plothist:
        fig,ax = plt.subplots(1,2,figsize=(10,4))
        ax[0].hist(imageAGNcrop.flatten(),bins=np.arange(np.min(imageAGNcrop),np.max(imageAGNcrop)))
        ax[1].hist(imageAGNcrop.flatten(),bins=np.arange(np.min(imageAGNcrop),20))
        [ax[i].set_title(['Whole intensity range',f'sky level: {float(sky):.4f}'][i]) for i in range(2)]
        [ax[i].set_xlabel('intensity') for i in range(2)]
        ax[0].set_ylabel('number of agns') 
    return sky

def profile_1D(semiA,image,PA=180,ell=0.5):
    """make 1D elliptical profiles"""
    # create guess ellipse
    pos0 = image.shape[0]//2
    geometry = EllipseGeometry(x0=pos0, y0=pos0, sma=semiA, eps=ell,
                               pa=PA * np.pi / 180.0)
    # load image and geometry
    ellipse = Ellipse(image, geometry)
    # do isophote fit
    isolist = ellipse.fit_image()
    return isolist

def plot_isophotes(ax,isolist,num_aper=10):
    """plot aperatures on image"""
    for sma in np.linspace(isolist.sma[0],isolist.sma[-1],num_aper):
        iso = isolist.get_closest(sma)
        x, y, = iso.sampled_coordinates()
        ax.plot(x, y, color='white',linewidth="0.5")
        
# def make_model_components(config,imshape):
#     """make model component images from best fit config"""
#     comp_names = config.functionLabelList()
#     comp_ims=[]
#     comp_pos = []
#     for i in range(len(config.getModelAsDict()['function_sets'])):
#         posX = config.getModelAsDict()['function_sets'][i]['X0']
#         posY = config.getModelAsDict()['function_sets'][i]['Y0']
#         functions = config.getModelAsDict()['function_sets'][i]['function_list']
#         for j in range(len(functions)):
#             if functions[j]['label'] =="bulge n=1" or functions[j]['label'] =="bulge n=4":
#                 functions[j]['parameters']['n'].append("fixed")
#             funcset_dict = {'X0': posX, 'Y0': posY, 'function_list': [functions[j]]}
#             model_dict = {'function_sets': [funcset_dict]}
#             model = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict)
#             imfit_fitter = pyimfit.Imfit(model,epsf)
#             comp_ims.append(imfit_fitter.getModelImage(shape=(imshape,imshape)))
#             comp_pos.append([posX[0],posY[0]])
#     return comp_ims, comp_pos, comp_names

def make_model_components(config,imshape):
    """make model component images from best fit config"""
    comp_names = config.functionLabelList()
    comp_ims=[]
    comp_pos = []
    # at each position in model
    for i in range(len(config.getModelAsDict()['function_sets'])):
        # get position and function sets 
        posX = config.getModelAsDict()['function_sets'][i]['X0']
        posY = config.getModelAsDict()['function_sets'][i]['Y0']
        functions = config.getModelAsDict()['function_sets'][i]['function_list']
        # for each function set
        for j in range(len(functions)):
            # fix pyimfit fault for fixed params
            if functions[j]['label'] =="bulge n=1":
                functions[j]['parameters']['n'] = [1, "fixed"]
            if functions[j]['label'] =="flat_sky":
                functions[j]['parameters']['I_sky'] = [functions[j]['parameters']['I_sky'][0],"fixed"]
            # create dictionary for each component 
            funcset_dict = {'X0': posX, 'Y0': posY, 'function_list': [functions[j]]}
            model_dict = {'function_sets': [funcset_dict]}
            # make fitter and component image
            model = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict)
            imfit_fitter = pyimfit.Imfit(model,epsf)
            # save to list
            comp_ims.append(imfit_fitter.getModelImage(shape=(imshape,imshape)))
            comp_pos.append([posX[0],posY[0]])
    return comp_ims, comp_pos, comp_names

def plot_model_components(comp_ims,comp_names,comp_pos,isolist_comps, colormap="ch:s=-.3,r=.6", plotIso=False,shrink_ratio=0.5):
    """plot 2D model components and check residual with model image"""
    clmap = sns.color_palette(colormap, as_cmap=True)
    ncom = len(comp_names)
    fig,ax = plt.subplots(nrows=1,ncols=ncom+1, figsize=(ncom*4,3))
    im = [ax[i].imshow(comp_ims[i],norm='symlog',cmap=clmap) for i in range(ncom)]
    [ax[i].text(0.05, 0.05, f"(x,y)=({comp_pos[i][0]:.1f},{comp_pos[i][1]:.1f})", transform=ax[i].transAxes, fontsize=8, color='k') for i in range(ncom-1)]
    [ax[i].set_title(comp_names[i]) for i in range(ncom)]
    im.append(ax[-1].imshow(np.sum(comp_ims[:-1],axis=0)-comp_ims[-1],norm='symlog',cmap=clmap))
    ax[-1].set_title("model-comps")
    [fig.colorbar(im[i], ax=ax[i], shrink=shrink_ratio).ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f')) for i in range(len(ax))]
    if plotIso:
        for i in range(len(isolist_comps)):
             plot_isophotes(ax[i],isolist_comps[i],num_aper=5)
    fig.tight_layout();

def plot_1isophote(ax,sma,isolist):
    """plot aperatures on image"""
    iso = isolist.get_closest(sma)
    x, y, = iso.sampled_coordinates()
    ax.plot(x, y, color='white',linewidth="0.3")

def pix_to_arcsec(imageFile,framelim):
    w = WCS(imageFile)
    # convert pixel to degree
    ra1,dec1 = w.pixel_to_world_values(0,0)
    ra2,dec2 = w.pixel_to_world_values(framelim,framelim)
    arcsec_per_pix = 0.16*u.arcsec
    return arcsec_per_pix, [ra1,dec1,ra2,dec2]

def surface_brightness(intensity, area, magZPT):
    return -2.5*np.log10(intensity/area)+ magZPT

def radial_plot_params(imageFile, framelim, isolist_data,isolist_comps,hdu_exp,z=0.2):
    # convert pixel to arcsec and kpc
    arcsec_per_pix, skycoords = pix_to_arcsec(imageFile,framelim)
    sma_arcsec = isolist_data.sma*arcsec_per_pix
    sma_kpc = (cosmo.angular_diameter_distance(z)*sma_arcsec.to('rad').value).to('kpc')
    # calculate isophote areas and find surface brightness
    areas = (np.sqrt((1-isolist_data.eps**2)*sma_arcsec**2)*np.pi*sma_arcsec).value
    magZPT = hdu_exp.header['MAGZP']
    mu_data = [surface_brightness(i,areas,magZPT) for i in [isolist_data.intens,isolist_data.intens-isolist_data.int_err,isolist_data.intens+isolist_data.int_err]]
    mu_models = [[surface_brightness(i,areas,magZPT) for i in [isolist_comps[j].intens,isolist_comps[j].intens-isolist_comps[j].int_err,isolist_comps[j].intens+isolist_comps[j].int_err]] for j in range(len(isolist_comps))]
    return sma_arcsec, sma_kpc, mu_data, mu_models, skycoords

def plot_everything(on,image,m,modelname,comp_names,fsr,sma_arcsec,sma_kpc,mu_data,mu_models,skycoords,colormap):
    colors = sns.color_palette(colormap, len(comp_names)+2)[1:]
    linestyles = ['-', '--', '-.', ':']
    #colors = sns.color_palette("colorblind", len(comp_names)+2)
    cmapp = sns.color_palette(colormap, as_cmap=True).reversed()
    # Create grid and add subplots
    fig = plt.figure(figsize=(14, 4),layout='tight')
    gs = gridspec.GridSpec(2, 4, height_ratios=[3, 1], width_ratios=[1.25,1.25,1,1.5],hspace=0.05,wspace=0.05)
    ax1 = fig.add_subplot(gs[:, 0],xlabel='RA (deg)',ylabel='DEC (deg)') 
    ax2 = fig.add_subplot(gs[:, 1],xticks=[],yticks=[])
    ax3 = fig.add_subplot(gs[:, 2],xticks=[],yticks=[])
    ax4a = fig.add_subplot(gs[0, 3]) 
    ax4b = fig.add_subplot(gs[1, 3])   
    # formatting ticks
    xticks = np.linspace(skycoords[0],skycoords[2],4)
    yticks = np.linspace(skycoords[1],skycoords[3],4)
    ax1.set_xticks(np.linspace(0,image.shape[0],4))
    ax1.set_yticks(np.linspace(0,image.shape[0],4))
    ax1.set_xticklabels([f'{x:.3f}' for x in xticks])
    ax1.set_yticklabels([f'{y:.3f}' for y in yticks],rotation=90)
    ax1.tick_params(direction='in')
    # plot 2d and colorbars
    ax = [ax1,ax2,ax3,ax4a,ax4b]
    im = [ax[i].imshow([image, m][i], norm='symlog',cmap=cmapp) for i in range(2)]
    im2 = ax[2].imshow(image-m,cmap=cmapp)
    fig.colorbar(im2,ax=ax[2],orientation='horizontal',location='bottom',pad=0.05)
    fig.colorbar(im[1],ax=[ax[0],ax[1]],orientation='vertical',location='right',shrink=0.5)
    [ax[i].set_title([on,f"Model:\n{modelname}",f'Residual,$\chi^2$={fsr:.0f}'][i]) for i in range(3)]
    # radial plot data
    ax[3].plot(sma_arcsec[1:],mu_data[0][1:],label="data",c=colors[-1])
    ax[3].fill_between(sma_arcsec[1:].value,mu_data[1][1:],mu_data[2][1:],color=colors[-1],alpha=0.5)
    # radial plot components
    [ax[3].plot(sma_arcsec[1:],mu_models[i][0][1:],label=comp_names[i],linestyle=linestyles[i],c=colors[i]) for i in range(len(comp_names)-1)]
    [ax[3].fill_between(sma_arcsec[1:].value,mu_models[i][1][1:],mu_models[i][2][1:],color=colors[i],alpha=0.5) for i in range(len(comp_names)-1)]
    ax[4].plot(sma_kpc[1:],mu_data[0][1:]-mu_models[-1][0][1:],c=colors[-2],linestyle="dashdot")
    ax[4].fill_between(sma_kpc[1:].value,mu_data[1][1:]-mu_models[-1][1][1:],mu_data[2][1:]-mu_models[-1][2][1:],color=colors[-2],alpha=0.5)
    ax[4].axhline(y=0,linestyle='--',c=colors[1],lw=1)
    # format ticks
    ax[3].invert_yaxis()
    ax[3].set_xlabel("R[arcsec]")
    ax[3].set_ylabel("$\mu$ [mag arcsec$^{-2}$]")
    ax[3].xaxis.set_label_position('top') 
    ax[3].xaxis.set_ticks_position('top') 
    ax[3].legend(fontsize=10,loc='upper right',bbox_to_anchor=(1.6, 1))
    [ax[i].set_xscale("log") for i in [3,4]]
    ax[4].set_xlabel("R[kpc]")
    ax[4].set_ylabel("$\Delta \mu$") 
    ax[4].set_ylim((-0.5,0.5))
    [ax.yaxis.set_label_position('right') for ax in [ax4a,ax4b]]
    [ax.yaxis.set_ticks_position('right') for ax in [ax4a,ax4b]];


def make_model_isophotes(isolist_data, comp_ims, comp_names, midf):
    isolist_comps=[]
    circ = [comp_names[i]=='psf' for i in range(len(comp_names))]
    for i in range(len(comp_ims)):
        isolist_ = []
        for iso in isolist_data[1:]:
            g = iso.sample.geometry
            ell = 0 if circ[i] else g.eps
            gn = EllipseGeometry(g.x0,g.y0, g.sma, ell, g.pa)
            sample = EllipseSample(comp_ims[i],g.sma,geometry=gn)
            sample.update()
            iso_ = Isophote(sample,0,True,0)
            isolist_.append(iso_)
        isolist = IsophoteList(isolist_)
        g = EllipseGeometry(midf,midf, 0.0, 0., 0.)
        sample = CentralEllipseSample(comp_ims[i], 0., geometry=g)
        fitter = CentralEllipseFitter(sample)
        center = fitter.fit()
        isolist.append(center)
        isolist.sort()
        isolist_comps.append(isolist)
    return isolist_comps

def make_data_isophotes(data,sma,midf):
    isolist_data = profile_1D(semiA=sma,image=data)
    # discard first isophote and make new
    isolist_data = isolist_data[1:]
    g = EllipseGeometry(midf,midf, 0.0, 0., 0.)
    sample = CentralEllipseSample(data, 0., geometry=g)
    fitter = CentralEllipseFitter(sample)
    center = fitter.fit()
    isolist_data.append(center)
    isolist_data.sort()
    return isolist_data

### fit 
read files

In [ ]:
on = "J0912+0148"
imageFile = glob.glob(os.path.expanduser("/home/insepien/research-data/agn-result/box/kpcbox/*"+on+"*.fits"))[0]
imageAGN = fits.getdata(imageFile)
# with open(os.path.expanduser("~/research-data/psf-results/psf_pkls/psf_"+on+".pkl"),"rb") as f:
#     d = pickle.load(f)
# epsf = d['psf'].data

epsf = fits.getdata("/home/insepien/dualAGN/scrap_notebooks/psf_j0912.fits")

ys,xs = find_highest_indices(imageAGN)
Imax = imageAGN.max()
itot=1500
framelim = imageAGN.shape[0]
midF=framelim//2

skylev = pd.read_pickle("../fit/sky41.pkl")#("/home/insepien/research-data/agn-result/box/skylev/sky_clean.pkl")
sky = skylev[on]#[0]
plt.imshow(imageAGN, norm='symlog')

In [ ]:
def _dofit_no_oversp(model, dataImage, psf, readnoise=0.22, expT=1, skylevel = 654.63, ncom=4, solver="NM",effgain=1):
    """do fit with not oversampled psf
       """
    fitter = pyimfit.Imfit(model,psf=psf)
    if float(ncom)==1.:
        fitter.loadData(dataImage,exp_time=expT, 
                    read_noise=readnoise, original_sky=skylevel,gain=effgain)
    else:
        fitter.loadData(dataImage,exp_time=expT, 
                    read_noise=readnoise, original_sky=skylevel,n_combined=ncom,gain=effgain)
    fitter.doFit(solver)
    fitConfig = fitter.getModelDescription()
    fitModelImage = fitter.getModelImage()
    fitResult = fitter.getFitResult()
    param_names = fitter.numberedParameterNames
    return fitConfig, fitModelImage, fitResult, param_names,fitter


Imax = imageAGN.max()
itot=1700
sersic_dict, psf_dict, gaussian_dict, flatsky_dict, flatbar_dict, exp_dict = makeModelDict(PA_ss=30, ell_ss=0.1, n_ss=1, I_ss=1, r_ss=20, Itot=itot,
                                                                     PA_lim=[0,360], ell_lim=[0.0,1.0], I_lim=[0.1,Imax],
                                                                     Iss_lim=[0.1,Imax], rss_lim=[0.1,framelim], Itot_lim=[0.1,1e4],
                                                                     sigma = 5, sigma_lim = [1,20], Isky = 2.5, Isky_lim =[0,10],
                                                                     h1=10,h2=10,h_lim=[0.1,framelim],alpha=0.1,alpha_lim=[0.1,framelim])
broken_exponentialParamsDict = {'PA': [0, 0,360], 'ell': [0.5, 0,1], 'I_0': [10.0, 0.0,Imax], 
                                'h1': [10, 0.1,framelim],'h2': [10, 0.1,framelim],'r_break': [10, 0.1,framelim],'alpha':[0.5,0,100]}
bexp_dict = {'name': "BrokenExponential", 'label': "bexp", 'parameters':broken_exponentialParamsDict}

GaussianRingAzParamsDict = {'PA': [40, 0,360], 'ell': [0.5, 0,1], 'A_maj':[100,0,framelim], 'A_min_rel':[0.5,0,1],
                          'R_ring':[10,0,100],'sigma_r':[5,0,100]}
gausR_dict = {'name': "GaussianRingAz", 'label': "GaussianRingAz", 'parameters': GaussianRingAzParamsDict}

exponentialParamDict = {'PA': [30, 0,360], 'ell': [0.5, 0,1], 'I_0': [1, 0.1,Imax], 'h': [1, 0.1,1e4]}
exp2_dict = {'name': "Exponential", 'label': "disk", 'parameters':exponentialParamDict}

flatbar = {'PA':[30,0,360], 'ell':[0.5,0,1],
            'deltaPA_max':[30,0,360], 'I_0':[1,0,Imax],
            'h1':[10,0.1,framelim], 'h2':[10,0.1,framelim,], 
            'r_break':[3,0,framelim], 'alpha':[0.1,0,framelim]}
flatbar0_dict = {'name': "FlatBar", 'label': "flat_bar", 'parameters':flatbar}

flatsky = {'I_sky': [sky,'fixed']}
flatsky_dict = {'name': "FlatSky", 'label': "flat_sky", 'parameters':flatsky}

sersicParamsDict = {'PA': [45, 0, 360], 'ell_bulge': [0.2, 0, 1],
                    'n': [1, 0.3,6],'I_e': [1, 0.0,Imax], 'r_e': [3, 0.0, framelim]}
sersic2_dict = {'name': "Sersic", 'label': "bulge 2", 'parameters': sersicParamsDict}

sersic1_dict = {'name': "Sersic", 'label': "bulge 1", 'parameters': sersicParamsDict}

sersic3ParamsDict = {'PA': [30, 0, 360], 'ell_bulge': [0.7, 0, 1],
                    'n': [1,  0.3,6],'I_e': [10, 0.0,Imax], 'r_e': [3, 0.0, framelim]}
sersic3_dict = {'name': "Sersic", 'label': "bulge 3", 'parameters': sersic3ParamsDict}

ext_sersicParamsDict = {'PA': [30, 0, 360], 'ell_bulge': [0.5, 0, 1],
                    'n': [1,  0.3,6],'I_e': [10, 0.0,Imax], 'r_e': [5, 0.0, framelim]}
sersic_ext_dict = {'name': "Sersic", 'label': "ext bulge", 'parameters': ext_sersicParamsDict}


funcset_dict_sersic1= {'X0': [xs[0],0,framelim], 'Y0': [ys[0],0,framelim], 
                    'function_list': [sersic1_dict,flatsky_dict]}
funcset_dict_sersic2= {'X0': [xs[1],0,framelim], 'Y0': [ys[1],0,framelim], 
                    'function_list': [psf_dict,sersic2_dict]}
funcset_dict_sersic3= {'X0': [75,0,framelim], 'Y0': [115,0,framelim], 
                    'function_list': [sersic2_dict]}




model_dict = {'function_sets': [funcset_dict_sersic1]}
model = pyimfit.ModelDescription.dict_to_ModelDescription(model_dict)

In [ ]:
fit_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_fit/"+on+".pkl"
# fit_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6(wrongBG)/masked_fit/"+on+".pkl"
with open(os.path.expanduser(fit_path),"rb") as f:
    d_fit = pickle.load(f)
modelname = "sersic+sersic(n1)+psf"
# d_fit['modelNames'][modelname].getModelAsDict()


In [ ]:
printparam = lambda name,val: [print(f"{n}: {v:.2f}") for n,v in zip(name,val)];
ind = np.where(np.array(list(d_fit['modelNames'].keys()))=="sersic+sersic(n1)+psf")[0][0]
printparam(d_fit['paramNames'][ind],d_fit['fitResults'][ind].params);

In [ ]:
i_explim = np.array([4.0, 0.1, 374.0625915527344])*np.exp(1.678)
r_explim = np.array([0.75, 0.1, 147.0])/1.678

In [ ]:
model = {'function_sets': [{'X0': [73.0, 0.0, 147.0],
   'Y0': [73.0, 0.0, 147.0],
   'function_list': [{'name': 'Sersic',
     'label': 'bulge',
     'parameters': {'PA': [40.0, 0.0, 360.0],
      'ell_bulge': [0.4, 0.0, 1.0],
      'n': [1.0, 0.3, 6.0],
      'I_e': [1.0, 0.1, 374.0625915527344],
      'r_e': [3.0, 0.1, 147.0]}},
    {'name': 'Exponential',
     'label': 'disk',
     'parameters': {'PA': [40.0, 0.0, 360.0],
      'ell': [0.4, 0.0, 1.0],
      'I_0': list(i_explim),
      'h': list(r_explim)}},
    {'name': 'PointSource',
     'label': 'psf',
     'parameters': {'I_tot': [1500.0, 0.1, 10000.0]}},
    {'name': 'FlatSky',
     'label': 'flat_sky',
     'parameters': {'I_sky': [-0.040774786844849586, 'fixed']}}]}]}

In [ ]:
mosfile = glob.glob(os.path.expanduser("~/raw-data-agn/mos-fits-agn/*"+on+"*.mos.fits"))[0]
expfile = glob.glob(os.path.expanduser("~/raw-data-agn/exp-fits-agn/*"+on+"*.exp.fits"))[0]
with fits.open(os.path.expanduser(mosfile)) as hdul:
    hdu0 = hdul[0]

sky_level = hdu0.header['BACKGND'] #fits:[e-/s] native pixels, pf: value should be in the same units as the data pixels

with fits.open(os.path.expanduser(expfile)) as hdul:
    hdu = hdul[0]
exptime= hdu0.header['EXPORG'] # fits: actual exp time, pf: total integration time
gain = hdu0.header['EGAIN'] #fits: e/du, pf: in electrons/ADU
noise=hdu.header['EFFRN'] #fits:[e-], pf: in electrons
numcom=hdu0.header['NCOADD'] #Number of Averaged Frames   
# Try for only 1 model
image = imageAGN
config0, modelIm0, fitRes0, pname0,fitter0  = _dofit_no_oversp(model,dataImage=image, psf=epsf,solver="LM",
                                                 readnoise=noise, expT=exptime, skylevel = sky_level, ncom=numcom,effgain=gain)
m = modelIm0
res = fitRes0.params
pname = pname0
fs = fitRes0.fitStat
fsr = fitRes0.fitStatReduced
comp_ims, comp_pos, comp_names = make_model_components(config0,imshape=image.shape[0])
comp_ims.append(m)
comp_names.append("model")
comp_pos.append([midF,midF])
[print(f"{pname0[i]}: {fitRes0.params[i]:.2f} ") for i in range(len(fitRes0.params))];
fig,ax = plt.subplots(1,3,figsize=(10,3),dpi=300)
im0 = ax[0].imshow(image,norm='symlog')
im1=ax[1].imshow(m,norm='symlog')
im2=ax[2].imshow(image-m,norm='symlog')
[fig.colorbar([im0,im1,im2][i],ax=ax[i],shrink=0.5) for i in range(3)]
ax[0].set_title(on)
ax[1].set_title("")
ax[2].set_title(f'residual, $\chi$:{fs:.2f},$\chi_r$:{fitRes0.fitStatReduced:.2f}')
fig.tight_layout();
print(len(fitRes0.params))

In [ ]:
isolist_comps=[]
plot_model_components(comp_ims,comp_names,comp_pos,isolist_comps)

In [ ]:
fig,ax = plt.subplots(1,2,figsize=(10,4))
im0=ax[0].imshow(comp_ims[0],norm='symlog')
im1=ax[1].imshow(m,norm='symlog')
[fig.colorbar([im0,im1][i],ax=ax[i],shrink=0.4) for i in range(2)]
fig.tight_layout();
ax[1].plot(comp_pos[0][0]-1,comp_pos[0][1]-1,"ro",markersize=2)
#ax[1].plot(comp_pos[2][0]-1,comp_pos[2][1]-1,"ro",markersize=2)
# ax[1].plot(80,135,"ro",markersize=1)
#ax[1].plot(40,40,"ro",markersize=1)
#[print(f"{pname0[i]}: {fitRes0.params[i]:.2f} ") for i in range(len(fitRes0.params))];

In [ ]:
modelname = 'sersic+psf,sersic+psf'
isolist_data = make_data_isophotes(image,20,midF)
isolist_comps = make_model_isophotes(isolist_data,comp_ims,comp_names,midF)  
plot_model_components(comp_ims,comp_names,comp_pos,isolist_comps)
sma_arcsec, sma_kpc, mu_data,mu_models,skycoords = radial_plot_params(imageFile, framelim,isolist_data,isolist_comps,hdu_exp=hdu0,z=0.2)
plot_everything(on,image,m,modelname,comp_names,fs,sma_arcsec,sma_kpc,mu_data,mu_models,skycoords,colormap="ch:s=-.3,r=.6")

In [ ]:
def save_data(modelname,image,model,configs,modelIms,fitResults,
              pnames,objectName, savepath):
    d = {}
    d[modelname] = model
    savedata = {}
    savedata['imageSS'] = image
    savedata['modelNames'] = d
    savedata['configs'] = configs
    savedata['modelImage'] = modelIms
    savedata['fitResults'] = fitResults
    savedata['paramNames'] = pnames
    pickle.dump(savedata,open(os.path.expanduser(savepath),"wb"))
    
# outDir =os.path.expanduser("~/research-data/agn-result/fit/fit_masked_n.3to6/nb_fit/"+on+"_nb.pkl")
# save_data(modelname,image=image, model=[model],configs=[config0],
#           modelIms=[modelIm0],fitResults=[fitRes0],pnames=[pname0],
#           objectName=on, savepath=outDir)



### tables for paper
note that this is for specific models: 1215 - sersic+psf,sersic+psf, 1222 - sersic,sersic

In [ ]:
################### j1215
from uncertainties import ufloat
import uncertainties.umath as um 
def make_paper_table(d_fit,d_comp,hdu0,ind,z):
    # get param vals and errs
    params = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind]['params']))
    errs = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind]['paramErrs']))
    # get comp total count 
    counts = [np.sum(i) for i in d_comp[modelname].comp_im]
    # convert to mag with errors
    magzp= hdu0.header['MAGZP']
    magzpe = hdu0.header['MAGZPE']
    comp_mag=[um.log10(ufloat(c,0)) * -2.5 + ufloat(magzp,magzpe) for c in counts[:-1]]
    # convert radius to phys distance
    platescale = hdu0.header['SCALE']
    physd = lambda r,z: ((r*platescale*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(z)).to(u.kpc).value
    re1 = physd(params['r_e_1'],z)
    re1_err = physd(errs['r_e_1'],z)
    re2 = physd(params['r_e_3'],z)
    re2_err = physd(errs['r_e_3'],z)
    # create df and convert to latex 
    fmt = lambda val, err: f"{val:.2f} $\pm$ {err:.2f}"
    keys = ['m(ser)','m(psf)','r(kpc)','n']
    comp1 = [fmt(comp_mag[0].n,comp_mag[0].s),fmt(comp_mag[1].n,comp_mag[1].s),fmt(re1,re1_err),fmt(params['n_1'],errs['n_1'])]
    comp2 = [fmt(comp_mag[2].n,comp_mag[2].s),fmt(comp_mag[3].n,comp_mag[3].s),fmt(re2,re2_err),fmt(params['n_3'],errs['n_3'])]
    print(pd.DataFrame([comp1,comp2],columns=keys).to_latex(column_format='|c|c|c|',float_format="%.3f"))

##################### j1222
def j1222_table(d_fit,d_comp):
    j1222params = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind]['params']))
    j1222errs = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind]['paramErrs']))
    # get comp total count 
    counts = [np.sum(i) for i in d_comp[modelname].comp_im]
    # convert to mag with errors
    magzp= hdu0.header['MAGZP']
    magzpe = hdu0.header['MAGZPE']
    comp_mag=[um.log10(ufloat(c,0)) * -2.5 + ufloat(magzp,magzpe) for c in counts[:-1]]
    # convert radius to phys distance
    platescale = hdu0.header['SCALE']
    z1222 = 0.172
    physd = lambda r,z: ((r*platescale*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(z)).to(u.kpc).value
    re1 = physd(j1222params['r_e_1'],z1222)
    re1_err = physd(j1222errs['r_e_1'],z1222)
    re2 = physd(j1222params['r_e_3'],z1222)
    re2_err = physd(j1222errs['r_e_3'],z1222)
    # create df and convert to latex 
    fmt = lambda val, err: f"{val:.2f} $\pm$ {err:.2f}"
    keys = ['m(ser)','r(kpc)','n']
    comp1 = [fmt(comp_mag[0].n,comp_mag[0].s),fmt(re1,re1_err),fmt(j1222params['n_1'],j1222errs['n_1'])]
    comp2 = [fmt(comp_mag[1].n,comp_mag[1].s),fmt(re2,re2_err),fmt(j1222params['n_3'],j1222errs['n_3'])]
    print(pd.DataFrame([comp1,comp2],columns=keys).to_latex(column_format='|c|c|c|',float_format="%.3f"))

############### models in appendix
modelnames = list(d_fit['modelNames'].keys())
paramnames = d_fit['paramNames']
nparams = [len(p) for p in paramnames]
intps=[]
mnames = []
for m,p in zip(modelnames,paramnames):
    mnames.append(m.replace("sersic", "S\'{e}rsic").replace("psf","PSF").replace("exp","Exponential").replace("n1","n=1"))
    intp = "Dual AGN" if m.count("psf")==1 else "Single AGN"
    intps.append(intp)

print(pd.DataFrame([mnames,nparams,intps],index=["mname", "N_dof", "Intepretation"], columns=np.arange(1,len(modelnames)+1)).T.to_latex(column_format="c|c"))

###########

In [ ]:
on = "J1140+0918"
modelname = "sersic+psf,sersic"

fit_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_fit/"+on+".pkl"
with open(os.path.expanduser(fit_path),"rb") as f:
    d_fit = pickle.load(f)
comp_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_comp/"+on+"_comp.pkl"
with open(os.path.expanduser(comp_path),"rb") as f:
    d_comp = pickle.load(f)
mosfile = glob.glob(os.path.expanduser("~/raw-data-agn/mos-fits-agn/*"+on+"*.mos.fits"))[0]
with fits.open(os.path.expanduser(mosfile)) as hdul:
    hdu0 = hdul[0]

getind = lambda name: np.where(np.array(list(d_fit['modelNames'].keys()))==name)[0][0]
ind = getind(modelname)

def make_paper_table(d_fit,d_comp,hdu0,ind,z,addflux=False):
    # get param vals and errs
    params = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind]['params']))
    print(params)
    errs = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind]['paramErrs']))
    # get comp total count 
    counts = [np.sum(i) for i in d_comp[modelname].comp_im]
    # remove background count
    counts = [c for c in counts[:-1] if c>0]
    # option to combine flux for ser+ser,ser models
    if addflux:
        counts = [counts[0]+counts[1], counts[2]]
    print(counts)
    # convert to mag with errors
    magzp= hdu0.header['MAGZP']
    magzpe = hdu0.header['MAGZPE']
    comp_mag=[um.log10(ufloat(c,0)) * -2.5 + ufloat(magzp,magzpe) for c in counts]
    # convert radius to phys distance
    platescale = hdu0.header['SCALE']
    physd = lambda r,z: ((r*platescale*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(z)).to(u.kpc).value
    re1 = physd(params['r_e_1'],z)
    re1_err = physd(errs['r_e_1'],z)
    re2 = physd(params['r_e_4'],z)
    re2_err = physd(errs['r_e_4'],z)
    # create df and convert to latex 
    fmt = lambda val, err: rf"${val:.2f} \pm {err:.2f}$"
    keys = ['m(ser)','r(kpc)','n',"e"]
    # comp1 = [fmt(comp_mag[0].n,comp_mag[0].s),fmt(re1,re1_err),fmt(params['n_1'],errs['n_1'])]
    print(fmt(comp_mag[0].n,comp_mag[0].s))
    comp2 = [fmt(comp_mag[1].n,comp_mag[1].s),fmt(re2,re2_err),fmt(params['n_4'],errs['n_4']),fmt(params['ell_bulge_4'],errs['ell_bulge_4'])]
    print(pd.DataFrame([comp2],columns=keys).to_latex(column_format='|c|c|c|',float_format="%.3f"))

make_paper_table(d_fit,d_comp, hdu0,ind,z=0.172,addflux=True)

### some 1-time-use functions to 
- cut fit.pkl from 17 to 15 models
- make fits with header
- make cutout of original size for extended boxes

In [ ]:
def reduce_to_15_models(on):
    """cut from 17 or more models to 15 models, i.e. remove bar models, for fit.pkl and comp.pkl"""
    fit_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit/"+on+".pkl"
    with open(os.path.expanduser(fit_path),"rb") as f:
        d_fit = pickle.load(f)
    key = list(d_fit.keys())
    # remove model dict
    for k in list(d_fit['modelNames'].keys())[15:]:
        del d_fit['modelNames'][k]
    # now remove from lists
    for k in key[2:]:
        try:
            d_fit[k] = d_fit[k][:15]
        except:
            print(k)
    if np.sum([len(d_fit[k])==15 for k in key[1:]]) == 5:
        print(f"{on} new fit.pkl len=15, saving fit pkl")
        pickle.dump(d_fit,open(os.path.expanduser("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit15/"+on+".pkl"),"wb"))

    comp_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit_comp/"+on+"_comp.pkl"
    with open(os.path.expanduser(comp_path),"rb") as f:
        d_comp = pickle.load(f)
    for k in list(d_comp.keys())[15:-2]:
        del d_comp[k]
    if len(d_comp)==17:
        print(f"{on} new comp.pkl len=15+2(data), saving comp pkl")
        pickle.dump(d_comp,open(os.path.expanduser("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_fit_comp15/"+on+"_comp.pkl"),"wb"))

def rewrite_image_fits_with_header(on):
    """make a new fits file for data image with header"""
    data= fits.getdata("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_image_SS/"+on+".fits")
    data0, header = fits.getdata(glob.glob(os.path.expanduser("~/research-data/agn-result/box/final_cut/"+on+"*"))[0],header=True)
    if data.shape==data0.shape:
        print(f"{on}: same size of {data.shape}, writing new file")
        fits.writeto("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_image_with_header/"+on+".fits",
                    data=data, header = header.copy(),overwrite=True)
    else:
        print(f"{on}: images not same size. need to resize")


def rebox_extended_boxes():
    """for 6 objects that extended box size from photutils, make cutouts again with original photutil sizes."""
    from astropy.coordinates import SkyCoord
    import astropy.units as u
    from astropy.nddata import Cutout2D
    # load coord catalog
    catalog = pd.read_csv('../cutouts/catalog.txt', names=['name', 'ra', 'dec'], delimiter='\s+')
    coords = [SkyCoord(ra=catalog['ra'].loc[i]*u.deg, dec=catalog['dec'].loc[i]*u.deg) for i in range(len(catalog))]
    catalog['coords'] = coords
    catalog.set_index("name",inplace=True)
    # get sizes from google doc
    onames = ["J0328-0710", "J0752+2019","J0918+1406","J0926+0724","J1204+0335","J1341-0049"]
    old_sizes = [100,100,80,100,200,150]
    sizes = [77,64,70,58,70,70]
    # create new cutouts
    for old,on,sz in zip(old_sizes, onames, sizes):
        try:
            d,h = fits.getdata("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/masked_image_with_header/"+on+".fits",header=True)
            if d.shape[0] == old:
                wcs = WCS(h)
                new_cutout = Cutout2D(d, position=catalog.loc[on,'coords'], size=(sz,sz), wcs=wcs)
                fits.writeto("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6/refit_boxsize/"+on+".fits", new_cutout.data, header=new_cutout.wcs.to_header())
                print(on, " done")
            else:
                print(f"{on}: old sizes don't match. {old} in doc vs {d.shape[0]} in fits")
        except:
            print(f"{on}: error")

### 1-time-use search for neighboring AGN
- in alpaka
- in SDSS

In [ ]:
from astropy.table import Table
from astropy.coordinates import SkyCoord

################## in alpaka
def find_neibor(name, nrow=3):
    """given object name, return boolean array of whether projected separation of nrow around object is under 110 kpc"""
    desig_mask = alpaka_agn['Desig']== name
    ind = alpaka_agn[desig_mask].index.values[0]
    # find max angle in deg at obj redshift for projected sep of 110 kpc
    thetaMax = ((110*u.kpc/cosmo.angular_diameter_distance(alpaka_agn['Z'][desig_mask].values[0])).to("")*u.rad).to(u.deg).value
    # check for angular distance around obj
    ocoord = SkyCoord(ra[desig_mask]*u.deg, dec[desig_mask]*u.deg)
    neibor_coord = [SkyCoord(ra[i]*u.deg, dec[i]*u.deg) for i in np.arange(ind-nrow,ind+nrow+1)]
    angsep = np.array([(ocoord.separation(o)[0]).to(u.deg).value for o in neibor_coord])
    dual_mask = [(angsep <= thetaMax) & (angsep != 0)]
    return dual_mask

# get sample names, alpaka
files = os.listdir("/home/insepien/research-data/agn-result/fit/fit_correct/masked_image_with_header/")
sizes = [fits.getdata("/home/insepien/research-data/agn-result/fit/fit_correct/masked_image_with_header/"+i).shape[0] for i in files]
onames = [f.split(".")[0] for f in files]
magel = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
# find the min and max cut out sizes
zs = [magel[magel['Desig'] == n]['Z'].values[0] for n in onames]
sz = np.array(sizes)
szkpc = ((sz*0.16*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(np.array(zs))).to(u.kpc)
print(f"Min size: {szkpc.min():.2f}, Max size: {szkpc.max():.2f}")

# load alpaka
alpaka = fits.getdata("/home/insepien/research-data/alpaka/ALPAKA_v1_withDes.fits")
alpaka = Table(alpaka).to_pandas()
# filter for agn only
non_agn_mask = alpaka["AGN_TYPE"] == -1
alpaka_agn = alpaka[~non_agn_mask]
alpaka_agn.sort_values(by=['RA','DEC'],inplace=True)
alpaka_agn.reset_index(inplace=True,drop=True)
# get ra and dec array
ra = alpaka_agn['RA'].values
dec = alpaka_agn['DEC'].values
# check for pairs under 110 kpc sep
sepmask = [find_neibor(name) for name in onames]
dualmask = [np.sum(i)==1 for i in sepmask]
# objects with neighboring AGN within 110 kpc projected sep
print(f"has neighbor AGN: {np.array(onames)[dualmask]}")

In [ ]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
magel = pd.read_pickle("/home/insepien/research-data/alpaka/alpaka_39fits.pkl")
magelCoords=[SkyCoord(r*u.deg,d*u.deg) for r,d in zip(magel['RA'],magel['DEC']*u.deg)]
magelZ = magel['Z'].values
# query sdss specobj in 110 kpc radius
sdss_query = []
for i in range(len(magel)):
    thetaMax = ((110*u.kpc/cosmo.angular_diameter_distance(magelZ[i])).to("")*u.rad).to(u.arcmin).value
    sdss_query.append(SDSS.query_region(magelCoords[i], radius=f'{thetaMax:.0f} arcmin',specobj_fields=['class','ra','dec','z'],data_release=18))

# check if there is multiple QSO search results 
multi_qso_mask = [(s['class'] == 'QSO').sum() > 1 for s in sdss_query]
multi_qso_magel = magel[multi_qso_mask]
multi_qso_ind = np.arange(len(sdss_query))[multi_qso_mask]
[sdss_query[m] for m in multi_qso_ind]

looking at .pkl fits

In [ ]:
on = "J1215+1344"
magel = pd.read_pickle("/home/insepien/research-data/alpaka/magellan/alpaka_39fits.pkl")
pix_to_phys = lambda pixlen: (((pixlen*0.16)*u.arcsec).to(u.rad).value*cosmo.angular_diameter_distance(magel[magel['Desig']==on].Z.values[0])).to(u.kpc)
imageFile = glob.glob(os.path.expanduser("/home/insepien/research-data/agn-result/box/kpcbox/*"+on+"*.fits"))[0]
# imageFile = glob.glob(os.path.expanduser("/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6(wrongBG)/masked_image_SS/*"+on+"*.fits"))[0]
imageAGN = fits.getdata(imageFile)
fit_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_fit/"+on+".pkl"
# fit_path = "/home/insepien/research-data/agn-result/fit/fit_masked_n.3to6(wrongBG)/masked_fit/"+on+".pkl"
with open(os.path.expanduser(fit_path),"rb") as f:
    d_fit = pickle.load(f)

getind = lambda name: np.where(np.array(list(d_fit['modelNames'].keys()))==name)[0][0]
printparam = lambda name,val: [print(f"{n}: {v:.2f}") for n,v in zip(name,val)]

modelname = "psf+sersic"
ind = getind(modelname)
printparam(d_fit['paramNames'][ind],d_fit['fitResults'][ind].params)
plt.imshow(imageAGN - d_fit['modelImage'][ind], norm='symlog')
plt.title(f"{on}, {modelname}")

def ang_sep(pos1,pos2,on):
    """calculate angular separation given dictionary of parameter names and values
        returns angular sep in arcsec and physical separation in kpc"""
    magel = pd.read_pickle("/home/insepien/research-data/alpaka/magellan/alpaka_39fits.pkl")
    z = magel[magel['Desig']==on]['Z'].values
    imageFile = glob.glob(os.path.expanduser("/home/insepien/research-data/agn-result/box/kpcbox/"+on+"*"))[0]
    w = WCS(imageFile)
    ra1,dec1 = w.pixel_to_world_values(pos1[0],pos1[1])
    ra2,dec2 = w.pixel_to_world_values(pos2[0],pos2[1])
    sep_rad = (angular_separation(ra1*u.deg,dec1*u.deg,ra2*u.deg,dec2*u.deg)).to(u.rad)
    sep_kpc = (cosmo.angular_diameter_distance(z)*sep_rad.value).to('kpc')
    return sep_rad.to(u.arcsec), sep_kpc
dparam = dict(zip(d_fit['paramNames'][ind],d_fit['fitResults'][ind].params))
# print("distance: ", ang_sep([dparam['X0_1'],dparam['Y0_1']], [dparam['X0_2'],dparam['Y0_2']],on))

comp_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_comp/"+on+"_comp.pkl"
with open(os.path.expanduser(comp_path),"rb") as f:
    d_comp = pickle.load(f)

counts = [np.sum(i) for i in d_comp[modelname].comp_im]
fx = np.array([c for c in counts[:-1] if c>0])
print("flux ratio: ", fx/fx.min())

In [ ]:
conf = d_fit['configs'][0].getModelAsDict()['options']
exptime = 26.199
geff = conf['GAIN']*4*exptime
sigma2 = (imageAGN + conf['ORIGINAL_SKY'])*exptime/geff + conf['NCOMBINED']*conf['READNOISE']**2/geff**2
w = 1/sigma2
mychi = np.sum(w*(imageAGN - d_fit['modelImage'][ind])**2)

print(f"my chi2: {mychi:.3f}")
print(f"imfit chi2: {d_fit['fitResults'][ind].fitStat:.3f}")

In [ ]:
def calchi(a,b):
    return d_fit['fitResults'][rank_ind[a]].fitStat - d_fit['fitResults'][rank_ind[b]].fitStat

on = "J1140+0918"
print(on)
fit_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer/highSky_fit/"+on+".pkl"
with open(os.path.expanduser(fit_path),"rb") as f:
    d_fit = pickle.load(f)
rank_ind = np.argsort([d.fitStat for d in d_fit['fitResults']])
modelnames = list (d_fit['modelNames'].keys())
for i in range(len(rank_ind)):
    print(f"rank {i} - aic {d_fit['fitResults'][rank_ind[i]].aic:.0f} - del {d_fit['fitResults'][rank_ind[0]].aic-d_fit['fitResults'][rank_ind[i]].aic:.0f} - {modelnames[rank_ind[i]]}")

j1215 spectra

In [ ]:
from matplotlib.patches import Circle
# load spectra
magel = pd.read_pickle("/home/insepien/research-data/alpaka/magellan/alpaka_39fits.pkl")
j1215 = magel[magel['Desig']=='J1215+1344']
f = fits.open("/home/insepien/research-data/pop-result/spec-1613-53115-0408.fits")
d = f[1].data
# load cutout fits for fiber plot
on = "J1215+1344"
imageFile = glob.glob(os.path.expanduser("/home/insepien/research-data/agn-result/box/kpcbox/*"+on+"*.fits"))[0]
imageAGN, header = fits.getdata(imageFile,header=True)
wcs = WCS(header)
fra_pix, fdec_pix = wcs.world_to_pixel_values(f[0].header['PLUG_RA'], f[0].header['PLUG_DEC'])
rad = 1.5/0.16 # sdss fibers are 3'' in diameter, divided by magellan plate scale

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(9,4),gridspec_kw={'width_ratios': [3,7],'height_ratios':[1]},dpi=300)

cmapp = sns.color_palette("ch:s=-.3,r=.6", as_cmap=True).reversed()
ax[0].imshow(imageAGN,norm='symlog',cmap=cmapp)
circ = Circle((fra_pix,fdec_pix), rad, fill=False,alpha=0.5)
ax[0].add_patch(circ)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[0].invert_yaxis()

ax[1].plot(10**d['loglam']/(1+j1215['Z'].values[0]),d['flux'],c=cmapp.colors[70],linewidth=0.5)
ax[1].set_xlabel("Rest wavelength [$\AA$]")
ax[1].set_ylabel("F$_{\lambda}$ [$10^{-17}~\rm{erg~s}^{-1}\rm{cm}^{-2} \AA^{-1}$]")
ax[1].text(0.05,0.90,f"{j1215['NAME'].values[0][7:21]}, $z=${j1215['Z'].values[0]:.3f}",transform=ax[1].transAxes)

fig.tight_layout();

In [ ]:
(cosmo.angular_diameter_distance(0.172)*(2.75*u.arcsec).to(u.rad).value).to(u.kpc)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(12,6))
ax[0].plot(10**d['loglam']/1.155,d['flux'],c=cmapp.colors[70],linewidth=1)
ax[0].set_xlim(5007-100,5007+100)
for wl, text in zip([4959,5007],['[OIII]4959','[OIII]5007']):
    ax[0].axvline(wl,alpha=0.2)
    ax[0].text(wl,25,text)


ax[1].plot(10**d['loglam']/1.155,d['flux'],c=cmapp.colors[70],linewidth=1)
ha_wl = 6564
ax[1].set_xlim(ha_wl-100,ha_wl+100)
ax[1].axvline(ha_wl,alpha=0.2)
ax[1].text(ha_wl,25,'HA')

[a.set_xlabel("Rest-frame wavelength (Angstrom)") for a in ax]
[a.set_ylabel("Flux, same unit") for a in ax]
fig.tight_layout();


In [ ]:
from scipy.optimize import curve_fit
fig,ax = plt.subplots(2,2,figsize=(10,6),sharey=True)

def gaussian_1d(x, amplitude, mean, std_dev):
        return amplitude * np.exp(-(x - mean)**2 / (2 * std_dev**2))

for wl,a,mul,lab,aax in zip([5007,6564],ax[1,:],j1215[['OIII_5007_FWHM',"HA_FWHM"]].values[0],['[OIII]','HA'],ax[0,:]):
    restwl = 10**d['loglam']/1.155
    # mask for 40 angstrom around line
    aa = 20
    mask57 = (restwl > wl-aa) & (restwl < wl+aa)
    # crop wl and flux
    cropwl = restwl[mask57]
    cropvel = (cropwl-wl)/wl*3e5
    # subtract continuum. Actually fitting cont with OIII gives 10.55, but 13 with HA, so assume 10 here
    cropflx = d['flux'][mask57] - 10
    # fit line w gaussian
    params, covariance = curve_fit(gaussian_1d, cropvel, cropflx, p0=[30,0,200])
    fitted_amplitude, fitted_mean, fitted_std_dev = params
    a.plot(cropvel, cropflx);
    a.plot(cropvel, gaussian_1d(cropvel, fitted_amplitude, fitted_mean, fitted_std_dev))
    a.set_xlabel("Velocity [km/s]")
    a.set_ylabel('Continuum subtracted flux')
    a.set_title(f"Fitted FWHM = {fitted_std_dev*2.355:.0f} km/s\n Mullaney FWHM: {mul:.0f} km/s")
    a.text(-700,25,lab)
    
    aax.step(cropwl,cropflx+10,where='mid')
    aax.set_xlabel("Wavelength [$\AA$]")
    aax.set_ylabel('flux')


fig.tight_layout();

# calculating sky error for uncertainty analysis

In [ ]:
from astropy.stats import sigma_clipped_stats
# load sky data
skydict = pd.read_pickle("/home/insepien/research-data/agn-result/box/skylev/sky_clean.pkl")
# sigma clip diff-boxsize-median at 2-sigma, get 1-sigma sky level
sky_std = np.array([sigma_clipped_stats(skydict[k][1][:,2], sigma=2)[-1] for k in skydict.keys()])
# save to new df
skydf = pd.DataFrame(skydict,index=['sky','data']).T
skydf['std'] = sky_std
skydf['pdiff'] = np.abs(sky_std/skydf['sky'])
fig,ax = plt.subplots(1,2,figsize=(7,3))
skydf['pdiff'].hist(grid='',ax=ax[0])
skydf[np.isin(skydf.index.values,dualnames)]['pdiff'].hist(grid='',ax=ax[1])
[a.set_xlabel("$\sigma$/sky level") for a in ax]
[a.set_title(tit) for a,tit in zip(ax,["full sample","dual+candidate"])]
fig.tight_layout();

# analyze sky uncer

In [ ]:
def get_flx_ratio(d_comp_,modelname_,combine=False):
    comp_ims = d_comp_[modelname_].comp_im
    counts = [np.sum(i) for i in comp_ims]
    fx = np.array([c for c in counts[:-1] if c>0])
    if combine:
        fx = np.array([fx[0]+fx[2],fx[1],fx[3]])
    fx_ratio = fx/fx.min()
    return fx_ratio

def get_flx(d_comp_,modelname_,combine=False):
    comp_ims = d_comp_[modelname_].comp_im
    counts = [np.sum(i) for i in comp_ims]
    fx = np.array([c for c in counts[:-1] if c>0])
    if combine:
        fx = np.array([fx[0]+fx[2],fx[1],fx[3]])
    return fx

def load_fid_fit(on):
    # load old fit
    fit_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_fit/"+on+".pkl"
    with open(os.path.expanduser(fit_path),"rb") as f:
        d_fit = pickle.load(f)
    comp_path = "/home/insepien/research-data/agn-result/fit/fit_kpcbox/kpcbox_comp/"+on+"_comp.pkl"
    with open(os.path.expanduser(comp_path),"rb") as f:
        d_comp = pickle.load(f)
    return d_fit, d_comp
    

def load_uncer_fit(fit_path_uncer,comp_path_uncer):  
    # load new fit
    with open(os.path.expanduser(fit_path_uncer),"rb") as f:
        d_fit_uncer = pickle.load(f)
    # uncer comp
    with open(os.path.expanduser(comp_path_uncer),"rb") as f:
        d_comp_uncer = pickle.load(f)
    return d_fit_uncer, d_comp_uncer

def get_model_ind(d_fit_, modelname_):
    return np.where(np.array(list(d_fit_['modelNames'].keys()))==modelname_)[0][0]

def cal_diff_percentage(old,new):
    return np.abs((old-new)/old*100)

def get_position_val(param_val, param_name):
    position_mask = [p[0] == 'X' or p[0]=='Y' for p in param_name]
    positions = param_val[position_mask]
    return positions

def get_flx_pos(on_, fit_path_uncer_, comp_path_uncer_, modelname_fid_, modelname_uncer_,combine_=False,ratio=False):
    d_fit_fid, d_comp_fid = load_fid_fit(on_)
    d_fit_uncer, d_comp_uncer = load_uncer_fit(fit_path_uncer_,comp_path_uncer_)
    # get model index
    ind_fid = get_model_ind(d_fit_fid,modelname_fid_)
    ind_uncer = get_model_ind(d_fit_uncer,modelname_uncer_)

    if ratio:
        #### get flux ratios
        fratio_fid = get_flx_ratio(d_comp_fid,modelname_fid_)
        fratio_uncer = get_flx_ratio(d_comp_uncer,modelname_uncer_,combine_)
    else:
        fratio_fid = get_flx(d_comp_fid,modelname_fid_)
        fratio_uncer = get_flx(d_comp_uncer,modelname_uncer_,combine_)
    #### get positions
    positions_fid = get_position_val(d_fit_fid['fitResults'][ind_fid].params,d_fit_fid['paramNames'][ind_fid])
    positions_uncer = get_position_val(d_fit_uncer['fitResults'][ind_uncer].params,d_fit_uncer['paramNames'][ind_uncer])
    
    return fratio_fid, fratio_uncer, positions_fid, positions_uncer



### look at flux to get mag changes

In [ ]:
on = "J1222-0007"
modelname_fid = "sersic,sersic"
modelname_uncer = "sersic,sersic"
flxd = []
posd = []
for uncertype in ['lowSky_fit','highSky_fit']:
    fit_path_uncer = os.path.join("/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer", uncertype, on+".pkl")
    comp_path_uncer = os.path.join("/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer",uncertype,on+"_comp.pkl")
    f_fid, f_uncer, pos_fid, pos_uncer = get_flx_pos(on, fit_path_uncer, comp_path_uncer, modelname_fid, modelname_uncer)
    magdif = -2.5*np.log10(f_fid/f_uncer)
    print(f"{uncertype}, diff in mag: {magdif}")
    # print(f_fid, f_uncer, pos_fid, pos_uncer)

### look at flux ratios pos changes

In [ ]:
dualnames = ['J1215+1344','J1222-0007','J0918+1207',"J0932+1611","J1140+0918"]
modelnames = ["sersic+psf,psf","sersic,sersic","sersic+sersic,sersic","sersic+sersic,sersic","sersic+psf,sersic"]


on = "J1215+1344"
modelname_fid = "sersic+psf,psf"
modelname_uncer = "sersic+psf,psf"
flxd = []
posd = []
for uncertype in ['lowSky_fit','highSky_fit']:
    fit_path_uncer = os.path.join("/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer", uncertype, on+".pkl")
    comp_path_uncer = os.path.join("/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer",uncertype,on+"_comp.pkl")
    fratio_fid, fratio_uncer, pos_fid, pos_uncer = get_flx_pos(on, fit_path_uncer, comp_path_uncer, modelname_fid, modelname_uncer)#,combine_= True)
    fx_diff = cal_diff_percentage(fratio_fid,fratio_uncer)
    pos_diff = cal_diff_percentage(pos_fid,pos_uncer)
    flxd.extend(fx_diff)
    posd.extend(pos_diff)
    
print(f"Fid model: {modelname_fid}")
print(f"Uncer model: {modelname_uncer}")
print(f"% diff in flux ratio: ({fx_diff[fx_diff!=0].min():.2e} , {fx_diff.max():.2e})")
print(f"% diff in positions: ({pos_diff.min():.2e} , {pos_diff.max():.2e})")

### look at R_e and n

In [ ]:
on = "J1222-0007"
d_fit, _ = load_fid_fit(on)
uncertype='lowSky_fit'
fit_path_uncer = os.path.join("/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer", uncertype, on+".pkl")
comp_path_uncer = os.path.join("/home/insepien/research-data/agn-result/fit/fit_kpcbox/fitUncer",uncertype,on+"_comp.pkl")
dfitu, dcompu = load_uncer_fit(fit_path_uncer, comp_path_uncer)

getind = lambda name,d_fit: np.where(np.array(list(d_fit['modelNames'].keys()))==name)[0][0]
printparam = lambda name,val: [print(f"{n}: {v:.4f}") for n,v in zip(name,val)]

modelname = "sersic,sersic"
ind = getind(modelname,dfitu)
print(uncertype)
printparam(dfitu['paramNames'][ind],d_fit['fitResults'][ind].params)
